<a href="https://colab.research.google.com/github/JCaballerot/Credit-Scoring/blob/main/%20CreditScoring/calibracion_y_seguimiento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5>Calibración de modelos
</font></h1>

---

### 1. Lectura de dataset

In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
pddf = pd.read_csv('loan_default_prediction.csv')
pddf.head()

### 2. Diagnóstico de calibración

In [ ]:
plt.figure(figsize=(6, 3))  # Tamaños

# gráfico de distribución
sns.histplot(pddf.prediction, color="lightskyblue")

# Muestra el gráfico
plt.xlabel("Probabilidad de Default")
plt.show()

In [ ]:
pddf[['prediction', 'Default']].describe()

In [7]:
calib = pddf.groupby('codmes').agg({'Default' :  'mean', 'prediction': 'mean'}).reset_index()


**Calibración mensual**

In [ ]:
# Graficar las series
plt.figure(figsize=(8, 4))  # Tamaños

# Convertir índices a strings para tratarlos como categóricos
calib.codmes = calib.codmes.astype(str)

plt.plot(calib.codmes, calib.Default,    label='rd', marker='o')
plt.plot(calib.codmes, calib.prediction, label='PD', marker='o')
plt.ylim(ymin=0, ymax=1)

plt.title('Calibración mensual')
plt.xlabel('Codmes')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


**Curva de calibración**

In [9]:
from sklearn.calibration import calibration_curve

In [10]:
probs = pddf.prediction
prob_true, prob_pred = calibration_curve(pddf.Default, probs, n_bins=6, strategy = 'quantile')

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms

fig, ax = plt.subplots()

plt.plot(prob_pred, prob_true, marker='o', linewidth=1, label='modelo sin calibrar')

# reference line, legends, and axis labels
plt.plot([0,1], [0,1], color='black', label="Calibración perfecta")
plt.xlim([0, 0.6])
plt.ylim([0, 0.6])

fig.suptitle('Calibration plot')
ax.set_xlabel('Predicted probability')
ax.set_ylabel('True probability in each bin')
plt.legend()
plt.show()

### 3. Calibración de modelo

In [ ]:
import statsmodels.api as sm

# Fit and summarize OLS model
mod = sm.Logit(pddf.Default, sm.add_constant(pddf.prediction))
res = mod.fit()

print(res.summary())

In [13]:
pddf['XB_CAL'] = 25.2886*pddf.prediction - 15.0513
pddf['PD_CAL'] = 1/(1 + np.exp(-pddf.XB_CAL))

In [ ]:
pddf[['Default', 'PD_CAL', 'prediction']].describe()

In [ ]:
plt.figure(figsize=(6, 3))  # Tamaños

# gráfico de distribución
sns.histplot(pddf.PD_CAL, color="lightskyblue")

# Muestra el gráfico
plt.xlabel("Probabilidad de Default")
plt.show()

**Calibración mensual**

In [ ]:
calib

In [ ]:
calib = pddf.groupby('codmes').agg({'Default' :  'mean', 'PD_CAL': 'mean'}).reset_index()

# Graficar las series
plt.figure(figsize=(8, 4))  # Tamaños

# Convertir índices a strings para tratarlos como categóricos
calib.codmes = calib.codmes.astype(str)

plt.plot(calib.codmes, calib.Default,    label='rd', marker='o')
plt.plot(calib.codmes, calib.PD_CAL,     label='PD_cal', marker='o')
plt.ylim(ymin=0, ymax=0.2)

plt.title('Calibración mensual')
plt.xlabel('Codmes')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


In [17]:

probs = pddf.PD_CAL
prob_true, prob_pred = calibration_curve(pddf.Default, probs, n_bins=6, strategy = 'quantile')


**Curva de calibración**

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.transforms as mtransforms

fig, ax = plt.subplots()

plt.plot(prob_pred, prob_true, marker='o', linewidth=1, label='modelo sin calibrar')

# reference line, legends, and axis labels
plt.plot([0,1], [0,1], color='black', label="Calibración perfecta")
plt.xlim([0, 0.3])
plt.ylim([0, 0.3])

fig.suptitle('Calibration plot')
ax.set_xlabel('Predicted probability')
ax.set_ylabel('True probability in each bin')
plt.legend()
plt.show()


<h1 align=center><font size = 5>Seguimiento de Modelos
</font></h1>

---


**Indice de estabilidad poblacional PSI**

Se utiliza para evaluar la estabilidad de una distribución de datos entre dos momentos en el tiempo o entre dos poblaciones diferentes.

Mide cuánto ha cambiado la distribución de datos (por ejemplo, puntuaciones de PD) entre dos puntos en el tiempo o entre dos grupos (como dos carteras de clientes).


In [19]:
# PASO1: Discretizar variables
from sklearn.preprocessing import KBinsDiscretizer

discretizer = KBinsDiscretizer(n_bins = 10,
                               encode = 'ordinal',
                               strategy = "quantile").fit(pddf[pddf.codmes == 202109][['PD_CAL']])


In [20]:
pddf[['PD_CAL_DISC']] = discretizer.transform(pddf[['PD_CAL']])

In [27]:
population = pddf.codmes.drop_duplicates().sort_values().tolist()

pddf_psi = pddf.pivot_table(index='PD_CAL_DISC', columns='codmes', values='ID', aggfunc='count').reset_index()
pddf_psi[population] = pddf_psi[population]/pddf_psi[population].sum()


In [ ]:
pddf_psi

In [48]:
%%capture
resume = pd.DataFrame({'codmes' : [],
                       'psi' : []})

for i in range(0, len(population)-1):
  psi = sum((pddf_psi[population[i+1]] - pddf_psi[population[i]])*np.log(pddf_psi[population[i+1]]/pddf_psi[population[i]]))
  resume = resume.append(pd.DataFrame({'codmes' : [population[i]],
                                       'psi' : [psi]}))


In [ ]:

# Graficar las series
plt.figure(figsize=(8, 4))  # Tamaños

# Convertir índices a strings para tratarlos como categóricos
resume.codmes = resume.codmes.astype(str)

plt.plot(resume.codmes, resume.psi, label='PSI', marker='o')
plt.ylim(ymin=0, ymax=0.04)

plt.title('PSI mensual')
plt.xlabel('Codmes')
plt.xticks(rotation=45)  # Rota las etiquetas del eje X a 45 grados
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

**límites de PSI**

> <b>Verde:</b> PSI < 0.1
>
> <b>Ambar:</b> PSI [0.1 - 0.2>
>
> <b>Rojo:</b> PSI >= 0.2



**Bandas de Confianza**

Proporciona un rango en el que es probable que se encuentre un valor desconocido. Son una extensión del concepto de intervalo de confianza a las predicciones realizadas por modelos, como regresiones, y se utilizan principalmente en la visualización de estimaciones y sus incertidumbres.

In [ ]:
#Intervalos de confianza

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

calib = pddf.groupby('codmes').agg({'Default': 'mean', 'PD_CAL': ['mean', 'std', 'count']}).reset_index()
calib.columns = ['codmes', 'Default', 'PD_CAL_mean', 'PD_CAL_std', 'count']

# Calcular bandas de confianza
z_90 = 1.64  # Valor z para 90% de confianza
z_95 = 1.96   # Valor z para 95% de confianza

calib['lower_90'] = calib['PD_CAL_mean'] - z_90 * calib['PD_CAL_std']*0.15
calib['upper_90'] = calib['PD_CAL_mean'] + z_90 * calib['PD_CAL_std']*0.15
calib['lower_95'] = calib['PD_CAL_mean'] - z_95 * calib['PD_CAL_std']*0.3
calib['upper_95'] = calib['PD_CAL_mean'] + z_95 * calib['PD_CAL_std']*0.3

# Graficar
fig, ax = plt.subplots(figsize=(8, 4))
calib.codmes = calib.codmes.astype(str)

ax.plot(calib.codmes, calib.Default, label='rd', marker='o', color = 'firebrick')
ax.plot(calib.codmes, calib.PD_CAL_mean, label='PD', marker='o', color = 'darkblue')
ax.fill_between(calib.codmes, calib['lower_95'], calib['upper_95'], color='yellow', alpha=0.2, label='95% Confianza')
ax.fill_between(calib.codmes, calib['lower_90'], calib['upper_90'], color='green', alpha=0.2, label='90% Confianza')
ax.set_ylim(ymin=0, ymax=0.20)

# Configurar el color
ax.set_facecolor((1, 0.9, 0.9))
ax.set_xlim(calib.codmes.min(), calib.codmes.max())

ax.set_title('Calibración mensual con Bandas de Confianza')
ax.set_xlabel('Codmes')
ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
#Intervalos OW

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

calib = pddf.groupby('codmes').agg({'Default': 'mean', 'PD_CAL': ['mean', 'std', 'count']}).reset_index()
calib.columns = ['codmes', 'Default', 'PD_CAL_mean', 'PD_CAL_std', 'count']


calib['lower_90'] = calib['PD_CAL_mean']*0.9
calib['upper_90'] = calib['PD_CAL_mean']*1.1
calib['lower_95'] = calib['PD_CAL_mean']*0.8
calib['upper_95'] = calib['PD_CAL_mean']*1.2

# Graficar
fig, ax = plt.subplots(figsize=(8, 4))
calib.codmes = calib.codmes.astype(str)

ax.plot(calib.codmes, calib.Default, label='rd', marker='o', color = 'firebrick')
ax.plot(calib.codmes, calib.PD_CAL_mean, label='PD', marker='o', color = 'darkblue')
ax.fill_between(calib.codmes, calib['lower_95'], calib['upper_95'], color='yellow', alpha=0.2, label='L2')
ax.fill_between(calib.codmes, calib['lower_90'], calib['upper_90'], color='green', alpha=0.2, label='L1')
ax.set_ylim(ymin=0, ymax=0.20)

# Configurar el color
ax.set_facecolor((1, 0.9, 0.9))
ax.set_xlim(calib.codmes.min(), calib.codmes.max())

ax.set_title('Calibración mensual con Bandas de OW')
ax.set_xlabel('Codmes')
ax.legend()
plt.tight_layout()
plt.show()

---
## Gracias por completar este laboratorio!